In [ ]:
import bqplot
import datetime
import dateutil
import ipywidgets
import ipyleaflet
import IPython.display
import pandas as pd

In [ ]:
import ee

ee.Initialize()

# Helper functions.

In [ ]:
"""Get a ipyleaflet compatible Tile Layer URL from an Earth Engine Image object."""
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

## ETCCDI Index #1 - Number of frost days (FD)

Annual count of days when the daily minimum temperature is below zero.

\begin{equation*}
FD_j = \sum_{i} TN_{ij}, \quad\text{if } TN_{ij} < 0^oC\label{a}\tag{1}
\end{equation*}

where $TN_{ij}$ is the daily minimum temperature on day $i$ in year $j$.

In [ ]:
def AddFrostDayBand(img):
    frost_day = img.select('tasmin').lt(273.15 + 0).rename('is_FD')
    return img.addBands(frost_day)

Create a reference to the NEX-GDDP dataset.

In [ ]:
gddp = ee.ImageCollection('NASA/NEX-GDDP')

Apply (map) the function to every image in the GDDP dataset.

In [ ]:
gddp = gddp.map(AddFrostDayBand)

Tally up the Frost Days for a given scenario, model, and year.

In [ ]:
frost_day_count = (gddp.select('is_FD')
    .filterMetadata('model', 'equals', 'CCSM4')
    .filterMetadata('scenario', 'equals', 'rcp45')
    .filterMetadata('year', 'equals', 2050)
).sum().uint16()

Plot results on an interactive map.

In [ ]:
map1 = ipyleaflet.Map(zoom=1, layout={'height':'300px'})
tile_url = GetTileLayerUrl(frost_day_count.visualize(min=0, max=365, bands= ['is_FD'], palette=['black', 'white']))
map1.add_layer(ipyleaflet.TileLayer(url=tile_url))
map1

# Interactive Viewer

In [ ]:
etccdi = ee.ImageCollection('users/tylere/ETCCDI/test3')

In [ ]:
model_list = [
    'ACCESS1-0',
    'BNU-ESM',
    'CCSM4',
    'CESM1-BGC',
    'CNRM-CM5',
    'CSIRO-Mk3-6-0',
    'CanESM2',
    'GFDL-CM3',
    'GFDL-ESM2G',
    'GFDL-ESM2M',
    'IPSL-CM5A-LR',
    'IPSL-CM5A-MR',
    'MIROC-ESM',
    'MIROC-ESM-CHEM',
    'MIROC5',
    'MPI-ESM-LR',
    'MPI-ESM-MR',
    'MRI-CGCM3',
    'NorESM1-M',
    'bcc-csm1-1',
    'inmcm4',
]
scenario_list = [
    'rcp45',
    'rcp85'
]

In [ ]:
# Define a annual count colormap.  
# #B3DE8E
sld = '''
<RasterSymbolizer>\
  <ChannelSelection>\
    <GrayChannel>\
      <SourceChannelName>FD</SourceChannelName>\
    </GrayChannel>\
  </ChannelSelection>\
  <ColorMap>\
    <ColorMapEntry color="#000000" quantity="0" />\
    <ColorMapEntry color="#F99B9B" quantity="10" />\
    <ColorMapEntry color="#389F34" quantity="200" />\
    <ColorMapEntry color="#A7CEE2" quantity="300" />\
    <ColorMapEntry color="#FFFFFF" quantity="365" />\
  </ColorMap>\
</RasterSymbolizer>
'''
legend = ipywidgets.HTML('''
    <form>
     <fieldset>
      <legend>Count:</legend>
      <pre style="text-align:center;background-color:#000000;color:white">0</pre>
      <pre style="text-align:center;background-color:#F99B9B">100</pre>
      <pre style="text-align:center;background-color:#389F34">200</pre>
      <pre style="text-align:center;background-color:#A7CEE2">300</pre>
      <pre style="text-align:center;background-color:#FFFFFF">365</pre>
     </fieldset>
    </form>
    ''',
    layout=ipywidgets.Layout(width='100px')
)

In [ ]:
debug_panel = ipywidgets.HTML('')

In [ ]:
def GetDataFrame(coords):
    
    param = 'FD'
    pnt = ee.Geometry.Point(coords)
    # Sample for a time series of values at the point.    
    geom_values = (etccdi
        .select(param)
        .filterMetadata('model', 'equals', model_select.value)
        .filterMetadata('scenario', 'equals', scenario_select.value)
        .getRegion(geometry=pnt, scale=10000)
    )
    geom_values_list = ee.List(geom_values).getInfo()
    # Convert to a Pandas DataFrame.
    header = geom_values_list[0]
    data = pd.DataFrame(geom_values_list[1:], columns=header)
    data['datetime'] = pd.to_datetime(data['time'], unit='ms', utc=True)
    data.set_index('time')
    data = data.sort_values('datetime')
    data = data[['datetime', param]]
    return data

In [ ]:
# Plot scales.
lc1_x = bqplot.DateScale(min=datetime.date(2001, 1, 1), max=datetime.date(2100, 1, 1))
lc2_y = bqplot.LinearScale()

# Plot type (mark).
lc2 = bqplot.Lines(
    x=[],
    y=[],
    scales={'x': lc1_x, 'y': lc2_y}, 
    display_legend=True,
)

# Plot axes.
x_ax1 = bqplot.Axis(label='Date', scale=lc1_x, num_ticks = 6, tick_format='%Y')
x_ay2 = bqplot.Axis(label='Freezing Days', scale=lc2_y, orientation='vertical')

# Declare the plot interactions.
br_intsel = bqplot.interacts.BrushIntervalSelector(scale=lc1_x, marks=[lc2])

# Create a figure.
fig = bqplot.Figure(
    marks=[lc2],
    axes=[x_ax1, x_ay2],
    layout={'height':'250px', 'width':'600px'},
    interaction=br_intsel
)

# Create a map widget with a drawing control.
map5 = ipyleaflet.Map(zoom=2, layout={'height':'270px', 'width':'600px'})
dc = ipyleaflet.DrawControl(polyline={}, polygon={})
map5.add_control(dc)

scenario_select = ipywidgets.Dropdown(
    options=scenario_list,
    description='Scenario:',
    disabled=False,
    layout=ipywidgets.Layout(width='250px')
)

model_select = ipywidgets.Dropdown(
    options=model_list,
    description='Model:',
    disabled=False,
    layout=ipywidgets.Layout(width='250px')
)

int_start_dp = ipywidgets.DatePicker(
    description='Start Date',
    disabled=True,
    layout=ipywidgets.Layout(width='250px')
)
int_end_dp = ipywidgets.DatePicker(
    description='End Date',
    disabled=True,
    layout=ipywidgets.Layout(width='250px')
)

# Create the event handlers for the map and plot.
def handle_draw(self, action, geo_json):
    # Get the selected coordinates from the map's drawing control.
    coords = geo_json['geometry']['coordinates']
    update_time_series()
dc.on_draw(handle_draw)

def ReplaceOverlayLayers(map_object, ee_image_object):
    for lyr in map_object.layers[1:]:
        map_object.remove_layer(lyr)
    tile_url = GetTileLayerUrl(ee_image_object)
    map_object.add_layer(ipyleaflet.TileLayer(url=tile_url))

def update_map():
    # Update the layer displayed on the map.
    filtered = (
        etccdi
            .select('FD')
            .filterMetadata('model', 'equals', model_select.value)
            .filterMetadata('scenario', 'equals', scenario_select.value)
            .filterDate(int_start_dp.value.isoformat(), int_end_dp.value.isoformat())
            .mean()
            .sldStyle(sld)
    )
    ReplaceOverlayLayers(map5, filtered)

def update_time_series():
    
    coords = dc.last_draw['geometry']['coordinates']
    new_df = GetDataFrame(coords)
    
    # Updatre the time series.
    lc2.x = new_df['datetime']
    lc2.y = new_df['FD']
    
    (t1_start, t1_end) = br_intsel.selected
    start_datetime = dateutil.parser.parse(t1_start)
    end_datetime = dateutil.parser.parse(t1_end)
    int_start_dp.value = start_datetime
    int_end_dp.value = end_datetime
    
def on_model_value_change(change):
    update_map()
    update_time_series()
model_select.observe(on_model_value_change, names='value')

def on_scenario_value_change(change):
    update_map()
    update_time_series()
scenario_select.observe(on_scenario_value_change, names='value')

def brush_selection_callback(change):
    update_time_series()
    update_map()
br_intsel.observe(brush_selection_callback, names=['selected'])

# Display the widgets.
space_time_viewer = ipywidgets.VBox(
    [
        ipywidgets.HBox([map5, legend]),
        ipywidgets.HBox(
            [
                fig,
                ipywidgets.VBox([
                    scenario_select,
                    model_select,
                    int_start_dp,
                    int_end_dp,
                    debug_panel
                ])
            ], layout=ipywidgets.Layout(align_content='center')
        ),
    ],
    align_self='stretch'
)

In [ ]:
space_time_viewer